In [2]:
import warnings
warnings.filterwarnings("ignore")

import requests
import numpy as np
import pandas as pd
import geopandas as gpd # GeoPandas library for spatial analytics
from shapely.geometry import shape
from shapely.geometry import shape

from datetime import datetime, timedelta

# Table

## [- Geocoding](#Geocoding)

1. [Address Data Cleaning](#Address-Data-Cleaning)
2. [Create cache-like database](#Create-cache-like-database)
3. [Geocoding for street segment](#Geocoding-for-street-segment)
4. [Get Geometry from LION](#Get-Geometry-from-LION)
5. [Create permit GeoDataFrame and Export it](#Create-permit-GeoDataFrame-and-Export-it)

## [- Count the Number of Permit](#Count-the-Number-of-Permit)
1. [Count Permit by Street Segments](#Count-Permit-by-Street-Segments)

# Geocoding

## Address Data Cleaning

In [3]:
df_film= pd.read_csv(r'../data/Film_Permits.csv')

In [4]:
#divide the multiple address string into multiple lists
df_film['OFT_Text'] = df_film['ParkingHeld'].str.split(',')
df_film['OFT_Text'] 

0       [SUTTON STREET between NORMAN AVENUE and NASSA...
1       [COVERT ST between WYCKOFF AVENUE and IRVING A...
2       [WHITE STREET between SEIGEL STREET and MOORE ...
3       [BARCLAY STREET between BROADWAY and CHURCH ST...
4       [WEST   13 STREET between 10 AVENUE and WASHIN...
                              ...                        
7362    [WEBSTER AVENUE between EAST  235 STREET and E...
7363    [PROVOST STREET between PAIDGE AVENUE and HURO...
7364    [EAST   11 STREET between 3 AVENUE and 4 AVENU...
7365    [HICKS STREET between HUNTINGTON STREET and WE...
7366    [PEARL STREET between PECK SLIP and BEEKMAN ST...
Name: OFT_Text, Length: 7367, dtype: object

## Create cache-like database

In [5]:
# create the cache-like database to store the result of query.
# If the address has been queried, then it won't be query again and saves time. 

data = {}
oft_lookup = {}
def getOFT(url):
    data = oft_lookup.get(url)
    if url not in oft_lookup:
        r = requests.get(url)
        data = r.json()
        oft_lookup[url] = data 
    return data    

## Geocoding for street segment

In [6]:
segidlookup=[]

def getGeom(row):
    boro = row['Borough']
    #print(boro)
    segids = []
    for oft in row['OFT_Text']:
        oft_1 = oft.strip()
        try:
            # split the address into on_street, from_street, to_street format, then the API can read
            on_street = oft_1.split('between')[0]
            from_street, to_street  = oft_1.split('between')[1].split('and')

            # query from geocoding service API
            url = f'https://www.locationservice.nyc/locationserviceapi//api/Block?OnStreet={on_street}&CrossStreetOne={from_street}&CrossStreetTwo={to_street}&Borough={boro}&BlockType=ExtendedStretch&ExtendedStretchType=Blockface&'
            data = getOFT(url)

            # extract the SegementID from JSON format
            if 'BlockFaceList' in data:
                for i in data['BlockFaceList']:
                    if 'SegmentID' in i:
                        segids.append(i['SegmentID'])
                        segidlookup.append({
                            'EventID': row['EventID'],
                            'SegmentID': i['SegmentID']
                        })
        except:

            continue          
    return segids
      
df_film['segids'] = df_film[:50].apply(getGeom, axis = 1)

## Get Geometry from LION

In [7]:
#create the dataset with eventID and Segment ID
df_segidlookup = pd.DataFrame(segidlookup)
df_segidlookup

,EventID,SegmentID
0,628327,0065898
1,629351,0048509
2,629319,0044431
3,629319,0044457
4,629319,0044453
...,...,...
261,560772,0032529
262,560772,0275252
263,560790,0048509
264,560879,0035675


In [10]:
#import LION shapfile
lion = gpd.read_file("../vector/lion_line")
lion_sel = lion[['OBJECTID','SegmentID','Join_ID','geometry']]
lion_sel.head(5)

,OBJECTID,SegmentID,Join_ID,geometry
0,1.0,0078126,2251001000000,"LINESTRING (1010964.447 241812.261, 1011265.49..."
1,2.0,0079796,2798401000000,"LINESTRING (1011576.687 255023.583, 1011335.15..."
2,3.0,0077356,2728001000000,"LINESTRING (1011600.676 239639.743, 1011785.79..."
3,4.0,0077356,21279502000000X,"LINESTRING (1011600.676 239639.743, 1011785.79..."
4,5.0,0077356,21279503000000X,"LINESTRING (1011600.676 239639.743, 1011785.79..."


In [11]:
#join LION to permit data
df_film_seg = df_film.merge(df_segidlookup,how='left',on='EventID')
df_film_lion = df_film_seg.merge(lion_sel,how='left',on='SegmentID')
df_film_lion.head(2)

,EventID,EventType,StartDateTime,EndDateTime,EnteredOn,EventAgency,ParkingHeld,Borough,CommunityBoard(s),PolicePrecinct(s),Category,SubCategoryName,Country,ZipCode(s),OFT_Text,segids,SegmentID,OBJECTID,Join_ID,geometry
0,628327,Shooting Permit,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,02/28/2022 11:13:50 AM,"Mayor's Office of Film, Theatre & Broadcasting",SUTTON STREET between NORMAN AVENUE and NASSAU...,Brooklyn,1,94,Television,Pilot,United States of America,11222,[SUTTON STREET between NORMAN AVENUE and NASSA...,[0065898],0065898,39195.0,3762801000000,"LINESTRING (1000675.780 203773.602, 1000590.62..."
1,628327,Shooting Permit,03/08/2022 07:00:00 AM,03/08/2022 07:00:00 PM,02/28/2022 11:13:50 AM,"Mayor's Office of Film, Theatre & Broadcasting",SUTTON STREET between NORMAN AVENUE and NASSAU...,Brooklyn,1,94,Television,Pilot,United States of America,11222,[SUTTON STREET between NORMAN AVENUE and NASSA...,[0065898],0065898,39196.0,36363001000000V,"LINESTRING (1000675.780 203773.602, 1000590.62..."


In [16]:
#Check the number/percentage of permits with no geometry
(df_film_lion['geometry'].isnull().sum(axis = 0))/len(df_film_lion)

0.03519414769052345

## Create permit GeoDataFrame and Export it

In [ ]:
#Turn permit dataframe to geodataframe
gdf_film_lion = gpd.GeoDataFrame(df_film_lion, crs="EPSG:4326").set_geometry('geometry')
gdf_film_lion_output = gdf_film_lion.drop(columns=['OFT_Text','segids'])

#Export the geodataframe to shapefile
gdf_film_lion_output.to_file('../output/film permit_city.shp')  

# Count the Number of Permit

## Count Permit by Street Segments

In [18]:
df_seg_count = gdf_film_lion_output[['EventID','SegmentID']].groupby('SegmentID').count()
df_seg_count = df_seg_count.rename(columns={'EventID':'counts'}).reset_index()
df_seg_count.sort_values(by='counts', ascending=False)

,SegmentID,counts
3016,0035681,349
2951,0035515,229
2955,0035520,220
2947,0035511,208
4976,0067749,182
...,...,...
462,0022412,1
4756,0065900,1
4759,0065937,1
4763,0065961,1
